<a href="https://colab.research.google.com/github/LEAF-BoiseState/Idaho-Mesonet-Design/blob/main/Idaho_PRISM_Clip_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
try:
    import wxee
except ImportError:
    print('wxee package not installed. Installing...')
    subprocess.check_call(['python','-m','pip','install','wxee'])
  #!pip install eemont

import ee
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

from google.colab import drive
drive.mount('/content/gdrive')

import wxee

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
prism_write_path = '_Idaho_TopoVeg'

idaho_norm91m_ppt = 'prism_ID_Norm91m_ppt'
idaho_norm91m_tmean = 'prism_ID_Norm91m_tmean'


In [ ]:
# Get the USGS HUC08 collection from the data catalog
us_states_tiger = ee.FeatureCollection('TIGER/2018/States')


id_outline = us_states_tiger.filter(ee.Filter.inList('STATEFP', ee.List(['16'])))

# Create a new map and add the watershed we will use to clip the EVI data
Map = emap.Map(center=(47.5, -116.25),zoom=5)

Map.addLayer(ee.Image().paint(id_outline, 0, 2), {}, 'Idaho')

Map.addLayerControl()
Map

Map(center=[47.5, -116.25], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…

In [ ]:
ID_prism_Norm91m = ee.ImageCollection('OREGONSTATE/PRISM/Norm91m').map(lambda img: img.clip(id_outline))



In [ ]:
ID_prism_Norm91m

In [ ]:
ID_prism_Norm91m_ppt = ID_prism_Norm91m.select('ppt').sum()
ID_prism_Norm91m_tmean = ID_prism_Norm91m.select('tmean').mean()


In [ ]:
ID_prism_Norm91m_tmean

In [ ]:
import geemap.colormaps as cm

Map2 = emap.Map(center=(47.5, -116.25),zoom=5)

precipitationVis = {
  'min': 0.0,
  'max': 2000.0,
  'palette': cm.get_palette('YlGnBu', n_class=8),
}

Map2.addLayer(ID_prism_Norm91m_ppt, precipitationVis, 'Mean Annual Precipitation [mm]')
Map2.addLayerControl()
Map2

Map(center=[47.5, -116.25], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…

In [ ]:
Map3 = emap.Map(center=(47.5, -116.25),zoom=5)

tempVis = {
  'min': 0.0,
  'max': 15,
  'palette': cm.get_palette('plasma', n_class=8),
}

Map3.addLayer(ID_prism_Norm91m_tmean, tempVis, 'Mean Annual Temperature [°C]')
Map3.addLayerControl()
Map3

Map(center=[47.5, -116.25], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…

In [ ]:
ID_prism_Norm91m_ppt = ID_prism_Norm91m_ppt.toDouble().unmask(-9999)
ID_prism_Norm91m_ppt = ID_prism_Norm91m_ppt.clip(id_outline.geometry().bounds())

ID_prism_Norm91m_tmean = ID_prism_Norm91m_tmean.toDouble().unmask(-9999)
ID_prism_Norm91m_tmean = ID_prism_Norm91m_tmean.clip(id_outline.geometry().bounds())

downConfig = {'scale': 928, "maxPixels": 1.0E13, 'driveFolder': prism_write_path }  # scale means resolution.

task1 = ee.batch.Export.image(ID_prism_Norm91m_ppt, idaho_norm91m_ppt, downConfig)
task1.start()
print(task1.status())

task2 = ee.batch.Export.image(ID_prism_Norm91m_tmean, idaho_norm91m_tmean, downConfig)
task2.start()
print(task2.status())


{'state': 'READY', 'description': 'prism_ID_Norm91m_ppt', 'creation_timestamp_ms': 1697520098583, 'update_timestamp_ms': 1697520098583, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'TPV65WNR5JYUSWFDMAU76LVL', 'name': 'projects/earthengine-legacy/operations/TPV65WNR5JYUSWFDMAU76LVL'}
{'state': 'READY', 'description': 'prism_ID_Norm91m_tmean', 'creation_timestamp_ms': 1697520099115, 'update_timestamp_ms': 1697520099115, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': '7MB2COAMPYMKQXBSIOATTAN4', 'name': 'projects/earthengine-legacy/operations/7MB2COAMPYMKQXBSIOATTAN4'}
